# Property Assessment Cleaning Notebook

A rough (and slightly hacky) way of doing some filtering on the assessments CSV.
This is because the raw asssessments file is **huge** (it takes several gigabytes of ram) and takes a *long* time to process here.
Essentially, this creates a CSV containing only the necessary rows.

The raw data can be obtained at: https://data.wprdc.org/dataset/property-assessments

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

2


In [2]:
# read in data
tobacco_data = pd.read_csv("tobacco_data.csv")

# Uh oh, the data is a mess! We need to do some cleanup...
# It turns out our data has a *lot* of extraneous spaces
# So, we can apply a lambda to every data piece to trim whitespace (if it's a string - not all entries are!)
# Not ideal, but it'll do!
tobacco_data = tobacco_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# filter out non pittsburgh entries
# data set uses "PITTSBURGH" and "PGH" - #consistency
tobacco_data = tobacco_data[tobacco_data.apply(lambda row: True if (row.City == "PITTSBURGH") or (row.City == "PGH") else False,axis=1)]

In [3]:
def clean_street(street):
    return street.replace("AVENUE", "AVE", 1).replace("STREET", "ST", 1).replace("BOULEVARD", "BLVD", 1).replace("HIGHWAY", "HWY", 1)

clean = pd.read_csv("raw_assessments.csv")

parcel_mappings = { }
for index, row in clean.iterrows():
    parcel_mappings[str(row['PROPERTYHOUSENUM']) + " " + row['PROPERTYADDRESS']] = row['PARID']

# loop over each entry of zip/number of occurrences
tobacco_locations = [ ]
for index, row in tobacco_data.iterrows():
    # First, we need to convert the zip code to a neighborhood
    street = row['Street1']
    street = clean_street(street)
    if street in parcel_mappings:
        tobacco_locations.append(street)
        continue
    else: # use backup street 2
        street = row['Street2']
        if street: # string is not empty
            street = clean_street(street)
            if street in parcel_mappings:
                tobacco_locations.append(street)
                continue

# not the best, but use a lambda to get the useful (matching) entries
clean = clean[clean.apply(lambda row: True if (str(row['PROPERTYHOUSENUM']) + " " + row['PROPERTYADDRESS']) in tobacco_locations else False,axis=1)]

# output to zip file (see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)
compression_opts = dict(method='zip',
                        archive_name='clean_assessments.csv')
clean.to_csv('clean_assessments.zip', index=False,
          compression=compression_opts) 

<ipython-input-3-ce5f64853f5a>:4: DtypeWarning: Columns (19,20,29,30,31,37,38,46,57,83) have mixed types. Specify dtype option on import or set low_memory=False.
  clean = pd.read_csv("raw_assessments.csv")
